In [1]:
import numpy as np
import torch
import botorch
import gpytorch

from matplotlib import pyplot as plt
%matplotlib widget

from utils import *

%load_ext autoreload
%autoreload 2

torch.set_default_dtype(torch.float64) # avoid matrix not positive semi-definite

In [2]:
# Step 1: init points
train_x, train_y, best_y = get_initial_data(10, ack, show_plot=False)

# Step 2; init gp model and fit
single_model, mll = init_gp_model(train_x, train_y)

# Step 3: Init acq func
ei = init_acq_func(single_model, best_y)

# Step 4: Generate candidates
candidates = gen_candidates(1, ei)

In [ ]:
for i in range(500):
    # Step 1: Evaluate candidates
    evaluation = evaluate_candidates(candidates, ack)
    train_x = torch.cat([train_x, candidates])
    train_y = torch.cat([train_y, evaluation])
    best_y = train_y.min()
    
    # Step 2: Init Gp Model
    single_model, mll = init_gp_model(train_x, train_y)
    
    # Step 3: Init acq func
    ei = init_acq_func(single_model, best_y)
    
    # Step 4: Acquire new candidate
    candidates = gen_candidates(1, ei)

    print(f"#{i+1:3,} | Best: {best_y} | ({from_unit_cube(candidates[0,0]):6,.02f}, {from_unit_cube(candidates[0,1]):6,.02f}) => {evaluation[0,0]:,.02f}")
    

#  1 | Best: 2.774470419096808 | ( 23.07,  -4.72) => 8.43
#  2 | Best: 2.774470419096808 | ( 23.26,  27.80) => 7.87
#  3 | Best: 2.774470419096808 | (-32.00,  32.00) => 9.87
#  4 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#  5 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#  6 | Best: 2.774470419096808 | ( 32.00, -32.00) => 10.81
#  7 | Best: 2.774470419096808 | ( 32.00, -18.87) => 10.81
#  8 | Best: 2.774470419096808 | ( 31.14,  32.00) => 9.19
#  9 | Best: 2.774470419096808 | (-32.00,  32.00) => 10.38
# 10 | Best: 2.774470419096808 | (-32.00,  32.00) => 10.81
# 11 | Best: 2.774470419096808 | ( 30.36, -32.00) => 10.81
# 12 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.10
# 13 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
# 14 | Best: 2.774470419096808 | (-32.00,  32.00) => 10.81
# 15 | Best: 2.774470419096808 | (-32.00,  32.00) => 10.81
# 16 | Best: 2.774470419096808 | (-32.00,  32.00) => 10.81
# 17 | Best: 2.774470419096808 | (-32.00,  32.00) => 10.81
#

/home/namit/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
/home/namit/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(
/home/namit/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/namit/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/namit/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/home/namit/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0

#195 | Best: 2.774470419096808 | ( -0.31, -15.57) => 10.81
#196 | Best: 2.774470419096808 | ( 32.00,  30.80) => 5.65
#197 | Best: 2.774470419096808 | ( 32.00, -31.17) => 10.24
#198 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.39
#199 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#200 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#201 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#202 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#203 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#204 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#205 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#206 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#207 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#208 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#209 | Best: 2.774470419096808 | (-32.00,  32.00) => 10.81
#210 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.81
#211 | Best: 2.774470419096808 | ( 32.00,  32.00) => 10.8

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(range(train_y.shape[0]), train_y[:,0])

Plotting the posterior distribution

In [ ]:
x = np.linspace(0,1,10)
y = np.linspace(0,1,10)
x_mesh, y_mesh = np.meshgrid(x,y)
x_mesh_denormalized = from_unit_cube(x_mesh)
y_mesh_denormalized = from_unit_cube(y_mesh)


mean_train_x = single_model.posterior(train_x).mean.detach().numpy()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter3D(train_x[:,0], train_x[:,1], mean_train_x.squeeze(), color="yellow")
ax.scatter3D(train_x[:,0], train_x[:,1], train_y.squeeze())


x = np.linspace(0,1,64)
y = np.linspace(0,1,64)
x_mesh, y_mesh = np.meshgrid(x,y)
mean_compatibale_coordinates = torch.tensor(  np.stack((x_mesh, y_mesh), axis=-1).reshape((4096,2))  )
mean = single_model.posterior(mean_compatibale_coordinates).mean.detach().numpy()

mean_mesh = mean.reshape((64,64))
ax.plot_surface(x_mesh, y_mesh, mean_mesh)


ax.plot_surface(x_mesh, y_mesh, ack(from_unit_cube(x_mesh), from_unit_cube(y_mesh)), color="yellow", alpha=0.2)

In [ ]:
print(x)
print(y)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# posterior surface plot
ax.plot_surface(x_mesh_denormalized, y_mesh_denormalized, mean)
ax.scatter(0,0,0, color="yellow")

# original function plot
x = np.linspace(-32,32,64)
y = np.linspace(-32,32,64)
x_ids, y_ids = np.meshgrid(x,y)
result = ack(x_ids,y_ids)
ax.plot_surface(x_ids, y_ids, result, color="yellow", alpha=0.2)

ax.set_xlabel("x")
ax.set_ylabel("y")



In [ ]:
mean.shape